In [111]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import numpy as np

In [117]:
df = pd.read_csv('raw.csv')
df = df.drop(['index'],axis=1)
df = df.loc[df['movement']=='out']
df['timestamp'] = pd.to_datetime(df['timestamp'])


In [118]:
df = df.drop(['id','location'],axis=1)

In [119]:
df['quantity_moved'] = df['quantity_moved'] *-1
df

,quantity_moved,material_name,timestamp,movement
18,1,1/2 HOSE BIBB,2024-03-06 13:07:00,out
66,3,4” Pvc Female Adaptor,2024-02-28 13:53:00,out
111,4,11/2 GALV STRAP,2024-03-13 12:43:00,out
113,4,2 GALV STRAP,2024-03-13 12:44:00,out
204,1,Badger 5 with cord,2024-03-12 16:35:00,out
...,...,...,...,...
13128,1,Dual Stop 1/2 X 3/8 X 1/4,2024-08-19 17:52:00,out
13130,1,2” Copper Male Adapter,2024-08-19 17:53:00,out
13131,10,1/2” CPVC Tee,2024-08-22 10:48:00,out
13133,10,1/2” Copper Pipe Type L,2024-08-22 10:49:00,out


In [120]:
df['month'] = df['timestamp'].dt.month
df['year'] = df['timestamp'].dt.year
df['date'] = df['month'].astype('str') + '-' + df['year'].astype('str')
df = df.drop(['timestamp','month','year'],axis=1)
df

,quantity_moved,material_name,movement,date
18,1,1/2 HOSE BIBB,out,3-2024
66,3,4” Pvc Female Adaptor,out,2-2024
111,4,11/2 GALV STRAP,out,3-2024
113,4,2 GALV STRAP,out,3-2024
204,1,Badger 5 with cord,out,3-2024
...,...,...,...,...
13128,1,Dual Stop 1/2 X 3/8 X 1/4,out,8-2024
13130,1,2” Copper Male Adapter,out,8-2024
13131,10,1/2” CPVC Tee,out,8-2024
13133,10,1/2” Copper Pipe Type L,out,8-2024


In [122]:
r = df.groupby(by=['date','material_name']).sum()
r.to_csv('linear_table.csv')
r = pd.read_csv('linear_table.csv')
r = r.rename(columns={'quantity_moved':'material_out'})


/tmp/ipykernel_17951/2533671510.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  r = df.groupby(by=['date','material_name']).sum()


In [123]:
r

,date,material_name,material_out
0,2-2024,"Fernco Proflex 2 in. Cast Iron, PVC and Steel...",1
1,2-2024,1-1/2” Copper P/P Cap,4
2,2-2024,1/2'' Copper P/P ST 90,18
3,2-2024,1” x 1” x 3/4” Copper P/P Tee,2
4,2-2024,3'' Fernco Flexible Coupling w/Stainless Steel...,1
...,...,...,...
1691,8-2024,Wolverine Brass Close Coupled Tank To Bowl Kit,3
1692,8-2024,Wolverine Brass Shower Head 3 1/2” Chrome 3 Se...,1
1693,8-2024,Wolverine Brass Soap/lotion Dispenser,1
1694,8-2024,Wolverine Brass Solid Brass Basket Strainer,2


In [128]:
l = r.groupby(by='material_name').mean().reset_index()
l = l.rename(columns={'material_out':'avg_out'})
l

/tmp/ipykernel_17951/1339236419.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  l = r.groupby(by='material_name').mean().reset_index()


,material_name,avg_out
0,"Fernco Proflex 2 in. Cast Iron, PVC and Steel...",1.0
1,#3 WAX RING W BOLTS,31.5
2,( Research 13),14.5
3,1 1/2'' CPVC Female Adapter,6.0
4,1 1/2'' Fernco Flexible Coupling w/Stainless S...,17.0
...,...,...
781,ZURN KEYS,2.0
782,Zawsalt,1.0
783,[Do Not Use] Instructional Test Item,2.0
784,"megapress jaws from 1/2 to 2""",1.0


In [130]:
l['max'] = np.ceil(l['avg_out'] * 1.2)
l

,material_name,avg_out,max
0,"Fernco Proflex 2 in. Cast Iron, PVC and Steel...",1.0,2.0
1,#3 WAX RING W BOLTS,31.5,38.0
2,( Research 13),14.5,18.0
3,1 1/2'' CPVC Female Adapter,6.0,8.0
4,1 1/2'' Fernco Flexible Coupling w/Stainless S...,17.0,21.0
...,...,...,...
781,ZURN KEYS,2.0,3.0
782,Zawsalt,1.0,2.0
783,[Do Not Use] Instructional Test Item,2.0,3.0
784,"megapress jaws from 1/2 to 2""",1.0,2.0


In [141]:
p = r.groupby(by='material_name').min().reset_index()
p = p.drop('date',axis=1)
p = p.rename(columns={'material_out':'min'})
p

,material_name,min
0,"Fernco Proflex 2 in. Cast Iron, PVC and Steel...",1
1,#3 WAX RING W BOLTS,6
2,( Research 13),5
3,1 1/2'' CPVC Female Adapter,6
4,1 1/2'' Fernco Flexible Coupling w/Stainless S...,17
...,...,...
781,ZURN KEYS,2
782,Zawsalt,1
783,[Do Not Use] Instructional Test Item,2
784,"megapress jaws from 1/2 to 2""",1


In [142]:
p.merge(l,on='material_name',how='outer')


,material_name,min,avg_out,max
0,"Fernco Proflex 2 in. Cast Iron, PVC and Steel...",1,1.0,2.0
1,#3 WAX RING W BOLTS,6,31.5,38.0
2,( Research 13),5,14.5,18.0
3,1 1/2'' CPVC Female Adapter,6,6.0,8.0
4,1 1/2'' Fernco Flexible Coupling w/Stainless S...,17,17.0,21.0
...,...,...,...,...
781,ZURN KEYS,2,2.0,3.0
782,Zawsalt,1,1.0,2.0
783,[Do Not Use] Instructional Test Item,2,2.0,3.0
784,"megapress jaws from 1/2 to 2""",1,1.0,2.0
